# 本节课目标：搭建神经网络，总结搭建八股

## 基于 Tensorflow 的 NN ：
用张量表示数据，计算图搭建神经网络，会话执行计算图，优化线上的权重，得到模型。

## 张量：多维数组

In [1]:
import tensorflow as tf
a = tf.constant([1.0,2.0])
b = tf.constant([2.0,3.0])
result = a + b
print(result)

Tensor("add:0", shape=(2,), dtype=float32)


### 计算图：搭建神经网络的计算过程，是承载一个或多个计算节点的一张图，只搭建网络，不运算。

In [2]:
x = tf.constant([[1.0,2.0]])
w = tf.constant([[3.0],[4.0]])
y = tf.matmul(x,w)
print(y)

Tensor("MatMul:0", shape=(1, 1), dtype=float32)


In [3]:
with tf.Session() as sess:
    print (sess.run(y))

[[ 11.]]


## 神经网络的参数

tf.Variable()

# 神经网络的搭建

1. 准备数据集，提取特征，作为输入喂给NN
2. 搭建NN结构，从输入到输出（先搭建计算图，再用会话执行）
3. 大量特征数据喂给NN，迭代优化NN参数
4. 使用训练好的模型预测和分类

## 前向传播

In [4]:
import tensorflow as tf
x = tf.placeholder(tf.float32,shape=(None,2))
w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

In [5]:
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

In [6]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(y,feed_dict={x:[[0.7,0.5]]}))

[[ 3.0904665]]


In [7]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(y,feed_dict={x:[[0.7,0.5],[0.2,0.3],[0.3,0.4],[0.4,0.5]]}))

[[ 3.0904665 ]
 [ 1.2236414 ]
 [ 1.72707319]
 [ 2.23050475]]


## 反向传播

损失函数loss
梯度下降 momentum优化 adam优化
均方误差MSE

# 例子(TF八股文，老师要求背下来)

### #0导入模块，生成模拟数据集。

In [8]:
#coding:utf-8

import tensorflow as tf
import numpy as np
BATCH_SIZE = 8
SEED = 23455

In [9]:
#基于seed产生随机数
rdm = np.random.RandomState(SEED)
#随机数返回32行2列的矩阵 表示32组 体积和重量 作为输入数据集
X = rdm.rand(32,2)
#从X这个32行2列的矩阵中 取出一行 判断如果和小于1 给Y赋值1 如果和不小于1 给Y赋值0 
#作为输入数据集的标签（正确答案） 
Y_ = [[int(x0 + x1 < 1)] for (x0, x1) in X]


### #1定义神经网络的输入、参数和输出,定义前向传播过程。

In [10]:
x = tf.placeholder(tf.float32, shape=(None, 2))
y_= tf.placeholder(tf.float32, shape=(None, 1))

w1= tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2= tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

### #2定义损失函数及反向传播方法。

In [11]:
loss_mse = tf.reduce_mean(tf.square(y-y_)) 
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss_mse)
#train_step = tf.train.MomentumOptimizer(0.001,0.9).minimize(loss_mse)
#train_step = tf.train.AdamOptimizer(0.001).minimize(loss_mse)

### #3生成会话，训练STEPS轮

In [12]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    # 输出目前（未经训练）的参数取值。
    print ("w1:\n", sess.run(w1))
    print ("w2:\n", sess.run(w2))
    print ("\n")
    
    # 训练模型。
    STEPS = 10000
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 32
        end = start + BATCH_SIZE
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y_[start:end]})
        if i % 500 == 0:
            total_loss = sess.run(loss_mse, feed_dict={x: X, y_: Y_})
            print("After %d training step(s), loss_mse on all data is %g" % (i, total_loss))
    
    # 输出训练后的参数取值。
    print ("\n")
    print( "w1:\n", sess.run(w1))
    print ("w2:\n", sess.run(w2))

w1:
 [[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]]
w2:
 [[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]


After 0 training step(s), loss_mse on all data is 5.13118
After 500 training step(s), loss_mse on all data is 0.429111
After 1000 training step(s), loss_mse on all data is 0.409789
After 1500 training step(s), loss_mse on all data is 0.399923
After 2000 training step(s), loss_mse on all data is 0.394146
After 2500 training step(s), loss_mse on all data is 0.390597
After 3000 training step(s), loss_mse on all data is 0.388336
After 3500 training step(s), loss_mse on all data is 0.386855
After 4000 training step(s), loss_mse on all data is 0.385863
After 4500 training step(s), loss_mse on all data is 0.385187
After 5000 training step(s), loss_mse on all data is 0.384719
After 5500 training step(s), loss_mse on all data is 0.384391
After 6000 training step(s), loss_mse on all data is 0.38416
After 6500 training step(s), loss_mse on all data is 0.383995
A